# 模型使用 
deepseek [https://python.langchain.com/api_reference/deepseek/index.html](https://python.langchain.com/api_reference/deepseek/index.html)

In [1]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="deepseek-v3",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    base_url="http://10.65.103.226:3000/v1",
    api_key="sk-BQaEAgXm4LtxQfFb7f709b2f91834050823b9a4432C081A5",
    # other params...
)

"""
from langchain_ollama import ChatOllama

llm = ChatOllama(
  model= "deepseek-r1:14b",
  base_url="http://10.230.66.59:11434",
  format="json"
)
"""



'\nfrom langchain_ollama import ChatOllama\n\nllm = ChatOllama(\n  model= "deepseek-r1:14b",\n  base_url="http://10.230.66.59:11434",\n  format="json"\n)\n'

提示词

In [2]:
from langchain.prompts import PromptTemplate
from langchain import hub

custom_prompt = PromptTemplate.from_template("""
你是一个旅游助手助手，可以回答旅游出行的相关问题。请遵循以下步骤：

1. 仔细分析用户问题
2. 根据问题类型选择合适工具
3. 使用工具处理问题
4. 用简洁的中文给出最终答案

请始终使用以下工具：
{tools}

历史对话记录：
{history}

问题：{input}
思考过程：""")

# --------------------- 步骤 4: 创建 Agent ---------------------
# 从 Hub 加载 ReAct 提示词（可自定义）
react_prompt = hub.pull("hwchase17/react")
react_prompt.pretty_print()

/root/python/notebook/.venv/lib/python3.10/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


tools

In [3]:
from langchain_core.tools import tool

import requests
import json

open_weather_key="6726d0210a47f4bacb1c134bed59ba82"

@tool
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询上海市天气，则loc参数需要输入'ShangHai'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    data = {
                "coord": {"lon": 116.4, "lat": 39.9},
                "weather": [
                    {
                    "id": 800,
                    "main": "Clear",
                    "description": "clear sky",
                    "icon": "01d"
                    }
                ],
                "main": {
                    "temp": 22.5,
                    "feels_like": 21.8,
                    "pressure": 1012,
                    "humidity": 45
                },
                "wind": {
                    "speed": 3.1,
                    "deg": 120
                },
                "name": loc
    }
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    """
    return json.dumps(data)

tools=[get_weather]

agent

In [4]:
from langchain.agents import create_react_agent

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_prompt  # 可替换为 custom_prompt
)


excutor

In [5]:
from langchain.agents import AgentExecutor

agent_excutor = AgentExecutor(
  agent=agent,
  tools=tools,
  verbose=True
)

In [6]:
agent_excutor.invoke(
    {
        "input": "北京的天气如何？"
    }
)



> Entering new AgentExecutor chain...
我需要查询北京的即时天气信息。  
Action: get_weather  
Action Input: BeiJing  
{"coord": {"lon": 116.4, "lat": 39.9}, "weather": [{"id": 800, "main": "Clear", "description": "clear sky", "icon": "01d"}], "main": {"temp": 22.5, "feels_like": 21.8, "pressure": 1012, "humidity": 45}, "wind": {"speed": 3.1, "deg": 120}, "name": "BeiJing  \n"}我现在知道了北京的天气信息。  
Final Answer: 北京的天气是晴朗的，温度为22.5摄氏度，体感温度为21.8摄氏度，湿度为45%，风速为3.1米/秒。

> Finished chain.


{'input': '北京的天气如何？',
 'output': '北京的天气是晴朗的，温度为22.5摄氏度，体感温度为21.8摄氏度，湿度为45%，风速为3.1米/秒。'}

In [7]:
agent_excutor.invoke(
    {
        "input": "北京和上海的天气一样吗？"
    }
)



> Entering new AgentExecutor chain...
我需要查询北京和上海的即时天气，然后比较它们的天气情况。  
Action: get_weather  
Action Input: Beijing  
{"coord": {"lon": 116.4, "lat": 39.9}, "weather": [{"id": 800, "main": "Clear", "description": "clear sky", "icon": "01d"}], "main": {"temp": 22.5, "feels_like": 21.8, "pressure": 1012, "humidity": 45}, "wind": {"speed": 3.1, "deg": 120}, "name": "Beijing  \n"}现在我需要查询上海的天气。  
Action: get_weather  
Action Input: Shanghai  

{"coord": {"lon": 116.4, "lat": 39.9}, "weather": [{"id": 800, "main": "Clear", "description": "clear sky", "icon": "01d"}], "main": {"temp": 22.5, "feels_like": 21.8, "pressure": 1012, "humidity": 45}, "wind": {"speed": 3.1, "deg": 120}, "name": "Shanghai  \n\n"}Thought: 我查询了北京和上海的天气，发现它们的天气描述、温度、湿度、风速等数据完全一致。  
Final Answer: 北京和上海的天气是一样的。

> Finished chain.


{'input': '北京和上海的天气一样吗？', 'output': '北京和上海的天气是一样的。'}